In [1]:
%pip install tkinter socketio-client threading sys

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)

[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for tkinter


In [2]:
import tkinter as tk
from tkinter import ttk, scrolledtext, messagebox
import socket
import threading
import sys

class NetworkSimulatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Simulador de Redes: Sockets TCP/IP")
        self.root.geometry("900x600")
        
        # Estilos para deixar a interface mais limpa
        self.style = ttk.Style()
        self.style.configure("TLabel", font=("Arial", 10))
        self.style.configure("TButton", font=("Arial", 10, "bold"))
        
        # Configuração do Socket do Servidor
        self.server_socket = None
        self.server_thread = None
        self.is_server_running = False
        self.client_socket = None

        # --- Layout Principal (Split Screen) ---
        # Esquerda: Cliente
        # Direita: Servidor
        paned_window = ttk.PanedWindow(root, orient=tk.HORIZONTAL)
        paned_window.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        self.client_frame = ttk.Labelframe(paned_window, text=" Lado do Cliente (Client) ", padding=10)
        self.server_frame = ttk.Labelframe(paned_window, text=" Lado do Servidor (Server) ", padding=10)

        paned_window.add(self.client_frame, weight=1)
        paned_window.add(self.server_frame, weight=1)

        self.setup_client_ui()
        self.setup_server_ui()

    # =========================================================================
    # LADO DO SERVIDOR (SERVER SIDE)
    # =========================================================================
    def setup_server_ui(self):
        # Controle do Servidor
        control_frame = ttk.Frame(self.server_frame)
        control_frame.pack(fill=tk.X, pady=5)

        ttk.Label(control_frame, text="Porta:").pack(side=tk.LEFT)
        self.server_port_var = tk.StringVar(value="5000")
        ttk.Entry(control_frame, textvariable=self.server_port_var, width=8).pack(side=tk.LEFT, padx=5)

        self.btn_start_server = ttk.Button(control_frame, text="Iniciar Servidor", command=self.start_server)
        self.btn_start_server.pack(side=tk.LEFT, padx=5)

        self.btn_stop_server = ttk.Button(control_frame, text="Parar", command=self.stop_server, state=tk.DISABLED)
        self.btn_stop_server.pack(side=tk.LEFT)

        # Log do Servidor
        ttk.Label(self.server_frame, text="Log do Servidor (Output):").pack(anchor=tk.W, pady=(10, 0))
        self.server_log = scrolledtext.ScrolledText(self.server_frame, height=20, bg="#e8f5e9", state=tk.DISABLED) # Verde claro
        self.server_log.pack(fill=tk.BOTH, expand=True, pady=5)

    def log_server(self, message):
        """Função auxiliar para escrever no log do servidor de forma thread-safe"""
        self.server_log.config(state=tk.NORMAL)
        self.server_log.insert(tk.END, f">> {message}\n")
        self.server_log.see(tk.END)
        self.server_log.config(state=tk.DISABLED)

    def start_server(self):
        port = int(self.server_port_var.get())
        
        try:
            # 1. Criação do Socket
            # AF_INET = IPv4, SOCK_STREAM = TCP
            self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            
            # Permite reutilizar a porta imediatamente se fechar e abrir rápido
            self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            
            # 2. Bind (Vincular IP e Porta)
            # '0.0.0.0' significa que aceita conexões de qualquer placa de rede deste PC
            self.server_socket.bind(('0.0.0.0', port))
            
            # 3. Listen (Começar a ouvir)
            self.server_socket.listen(5) # Fila de até 5 conexões
            
            self.is_server_running = True
            self.log_server(f"Servidor INICIADO na porta {port}")
            self.log_server("Aguardando conexões (Listening)...")
            
            # Atualiza Botões
            self.btn_start_server.config(state=tk.DISABLED)
            self.btn_stop_server.config(state=tk.NORMAL)

            # Inicia thread para aceitar conexões sem travar a interface
            self.server_thread = threading.Thread(target=self.accept_connections, daemon=True)
            self.server_thread.start()

        except Exception as e:
            messagebox.showerror("Erro", f"Não foi possível iniciar o servidor: {e}")

    def accept_connections(self):
        while self.is_server_running:
            try:
                # 4. Accept (Bloqueia até alguém conectar)
                # conn é um NOVO socket dedicado a esse cliente específico
                conn, addr = self.server_socket.accept()
                
                # Atualiza GUI na thread principal
                self.root.after(0, lambda: self.log_server(f"Conexão ACEITA de {addr}"))
                
                # Cria uma thread para tratar este cliente específico
                client_handler = threading.Thread(target=self.handle_client, args=(conn, addr), daemon=True)
                client_handler.start()
            except OSError:
                break

    def handle_client(self, conn, addr):
        with conn:
            while self.is_server_running:
                try:
                    # 5. Recv (Receber dados)
                    data = conn.recv(1024) # Buffer de 1024 bytes
                    if not data:
                        break # Cliente desconectou
                    
                    text = data.decode('utf-8')
                    self.root.after(0, lambda: self.log_server(f"Recebido de {addr[0]}: '{text}'"))
                    
                    # 6. Processamento e Resposta (Echo Server)
                    response = f"Servidor recebeu: {text.upper()}"
                    conn.sendall(response.encode('utf-8'))
                    
                except ConnectionResetError:
                    break
            self.root.after(0, lambda: self.log_server(f"Cliente {addr} desconectado."))

    def stop_server(self):
        self.is_server_running = False
        if self.server_socket:
            self.server_socket.close()
        self.log_server("Servidor PARADO.")
        self.btn_start_server.config(state=tk.NORMAL)
        self.btn_stop_server.config(state=tk.DISABLED)

    # =========================================================================
    # LADO DO CLIENTE (CLIENT SIDE)
    # =========================================================================
    def setup_client_ui(self):
        # Configuração de Conexão
        conn_frame = ttk.Frame(self.client_frame)
        conn_frame.pack(fill=tk.X, pady=5)

        ttk.Label(conn_frame, text="IP Destino:").grid(row=0, column=0, padx=5, sticky=tk.W)
        self.target_ip_var = tk.StringVar(value="127.0.0.1")
        ttk.Entry(conn_frame, textvariable=self.target_ip_var, width=15).grid(row=0, column=1, padx=5)

        ttk.Label(conn_frame, text="Porta:").grid(row=0, column=2, padx=5, sticky=tk.W)
        self.target_port_var = tk.StringVar(value="5000")
        ttk.Entry(conn_frame, textvariable=self.target_port_var, width=8).grid(row=0, column=3, padx=5)

        self.btn_connect = ttk.Button(conn_frame, text="Conectar", command=self.connect_to_server)
        self.btn_connect.grid(row=0, column=4, padx=10)

        # Envio de Mensagem
        msg_frame = ttk.LabelFrame(self.client_frame, text=" Enviar Mensagem ", padding=10)
        msg_frame.pack(fill=tk.X, pady=10)
        
        self.msg_var = tk.StringVar()
        entry_msg = ttk.Entry(msg_frame, textvariable=self.msg_var)
        entry_msg.pack(fill=tk.X, side=tk.LEFT, expand=True, padx=(0, 5))
        entry_msg.bind("<Return>", lambda event: self.send_message()) # Enviar com Enter

        self.btn_send = ttk.Button(msg_frame, text="Enviar", command=self.send_message, state=tk.DISABLED)
        self.btn_send.pack(side=tk.LEFT)

        # Log do Cliente
        ttk.Label(self.client_frame, text="Log do Cliente (Status):").pack(anchor=tk.W)
        self.client_log = scrolledtext.ScrolledText(self.client_frame, height=15, bg="#e3f2fd", state=tk.DISABLED) # Azul claro
        self.client_log.pack(fill=tk.BOTH, expand=True, pady=5)

    def log_client(self, message):
        self.client_log.config(state=tk.NORMAL)
        self.client_log.insert(tk.END, f">> {message}\n")
        self.client_log.see(tk.END)
        self.client_log.config(state=tk.DISABLED)

    def connect_to_server(self):
        ip = self.target_ip_var.get()
        port = int(self.target_port_var.get())

        try:
            # 1. Criação do Socket Cliente
            self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.client_socket.settimeout(5) # Timeout de 5s para não travar
            
            self.log_client(f"Tentando conectar a {ip}:{port}...")
            
            # 2. Connect (Handshake de 3 vias acontece aqui)
            self.client_socket.connect((ip, port))
            
            self.log_client("CONECTADO com sucesso!")
            self.btn_connect.config(state=tk.DISABLED)
            self.btn_send.config(state=tk.NORMAL)
            
            # Thread para ouvir respostas do servidor
            threading.Thread(target=self.receive_from_server, daemon=True).start()

        except Exception as e:
            self.log_client(f"Erro ao conectar: {e}")
            messagebox.showerror("Erro de Conexão", f"Não foi possível conectar: {e}")

    def send_message(self):
        if not self.client_socket:
            return
        
        msg = self.msg_var.get()
        if not msg:
            return

        try:
            # 3. Send (Enviar dados)
            # É necessário converter string para bytes (encode)
            self.client_socket.sendall(msg.encode('utf-8'))
            self.log_client(f"Enviado: {msg}")
            self.msg_var.set("") # Limpar campo
        except Exception as e:
            self.log_client(f"Erro ao enviar: {e}")

    def receive_from_server(self):
        while True:
            try:
                # 4. Recv (Aguardar resposta)
                data = self.client_socket.recv(1024)
                if not data:
                    self.log_client("Conexão fechada pelo servidor.")
                    break
                text = data.decode('utf-8')
                self.root.after(0, lambda: self.log_client(f"Resposta do Server: {text}"))
            except OSError:
                break
        
        # Resetar UI se cair a conexão
        self.root.after(0, self.reset_client_ui)

    def reset_client_ui(self):
        self.btn_connect.config(state=tk.NORMAL)
        self.btn_send.config(state=tk.DISABLED)
        if self.client_socket:
            self.client_socket.close()
            self.client_socket = None

if __name__ == "__main__":
    root = tk.Tk()
    app = NetworkSimulatorApp(root)
    root.mainloop()

Exception in thread Thread-10 (receive_from_server):
Traceback (most recent call last):
  File "C:\Users\bruno\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\bruno\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\bruno\AppData\Local\Temp\ipykernel_14944\3302821167.py", line 244, in receive_from_server
  File "C:\Users\bruno\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 873, in after
    name = self._register(callit)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bruno\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1604, in _register
    self.tk.createcommand(name, f)
RuntimeError: main thread is not in main loop
